# 데이터 크롤링

- [awesome-devblog : feeds](https://awesome-devblog.now.sh/api/korean/people/feeds)

In [1]:
import requests, json
import os, re, csv
import pandas as pd
from tqdm import trange

In [2]:
DATA_PATH = './data'
DATA_URL = 'https://awesome-devblog.now.sh/api/korean/people/feeds'
DOCUMENTS_PATH = './data/documents.tsv'

In [168]:
def getTotal():
    """
    전체 문서 개수 조회
    """
    res = requests.get(DATA_URL, { 'size': 1 }).json()
    return res['total'][0]['count']

def reqDoc(page, size):
    """
    문서 조회
    """
    page += 1
    params = {
        'sort': 'date.asc',
        'page': page,
        'size': size
    }
    return requests.get(DATA_URL, params).json()

def reqDocs(size, start_page=0):
    """
    전체 문서 조회
    """
    total = getTotal()
    if size > 5000: size = 5000
    total_req = round(total/size + 0.5)
    docs = None
    for i in trange(start_page, total_req):
        doc = reqDoc(i, size)
        if docs == None:
            docs = doc
        else:
            docs['data'] += doc['data']
    return docs

def updateDocs():
    """
    신규 문서 추가
    """
    size = 5000
    # imgUr(key 자체가 경우가 존재), postImage 배제
    keys = ['_id', 'title', 'link', 'date', 'description', 'author', 'tags', 'count']
    if not os.path.isfile(DOCUMENTS_PATH):
        # 데이터가 없는 경우, 전체 데이터를 불러서 신규 생성
        docs = reqDocs(size)
        with open(DOCUMENTS_PATH, 'w') as tsv_file:
            tsv_writer = csv.writer(tsv_file, delimiter='\t')
            tsv_writer.writerow(['_no', '_label'] + keys)
            for i, doc in enumerate(docs['data']):
                tsv_writer.writerow([i + 1, -1] + [re.sub('\t', '', str(doc[k])) for k in keys])
    else:
        # 기존 데이터가 있는 경우, 신규 데이터를 추가
        num_new_docs = 0
        tsv = pd.read_csv(DOCUMENTS_PATH, delimiter='\t')
        old_total = tsv.tail(1)['_no'].values[0]
        docs = reqDocs(size, old_total // size)
        no = old_total + 1
        with open(DOCUMENTS_PATH, 'a') as tsv_file:
            tsv_writer = csv.writer(tsv_file, delimiter='\t')
            for i, doc in enumerate(docs['data']):
                if doc['_id'] not in tsv._id.unique():
                    num_new_docs += 1
                    tsv_writer.writerow([no + i, ''] + [re.sub('\t', '', str(doc[k])) for k in keys])
        
        if num_new_docs == 0:
            print('문서가 이미 최신 상태입니다.')
        else:
            print(f'신규 문서 {num_new_docs}개 추가')

def tempSetLabel():
    """
    TEMP : 기존 라벨링한거 기존꺼에 반영
    """
    ntsv = pd.read_csv(DOCUMENTS_PATH, delimiter='\t')
    btsv = pd.read_csv('/Users/nero/Project/DocClassification/data/labeled.tsv', delimiter='\t')
    for index, row in btsv[:10000].iterrows():
        link = row.link
        title = row.title
        label = int(row.label)
        if not len(ntsv.loc[ntsv.title.str.strip() == title.strip()]) and not len(ntsv.loc[ntsv.link == link]):
            print(row.title)
        elif len(ntsv_title):
            ntsv.loc[ntsv.title.str.strip() == title.strip(), '_label'] = label
        elif len(ntsv_link):
            ntsv.loc[ntsv.link == link, '_label'] = label
        else:
            print('??')
    return ntsv

def getDocs():
    return pd.read_csv(DOCUMENTS_PATH, delimiter='\t')

In [167]:
updateDocs()

100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


신규 문서 0개 추가


In [165]:
getDocs()

,Unnamed: 0,_no,_label,_id,title,link,date,description,author,tags,count
0,0,1,-1,5de4f0195f4be70004fa640b,Fixed,https://iamsang.com/fixed/,0000-12-30T00:00:00.000Z,NaN,이상현,[],0
1,1,2,-1,5ceb240799e0450004600983,Materials,https://mrchypark.github.io/material/,0000-12-30T00:00:00.000Z,"직접 제작한 자료를 공유합니다. 부족한 부분도 많고, 잘못된 부분이 있을 수도 있습...",박찬엽,[],1
2,2,3,-1,5c66c39491b4130004e506bf,About,/about/,0000-12-30T00:00:00.000Z,English Ver. R과 docker를 사랑하는 데이터 관련 종사자입니다. Te...,박찬엽,[],0
3,3,4,-1,5c66c39491b4130004e506c3,Support,/support/,0000-12-30T00:00:00.000Z,블로그가 도움이 되셨나요? 저는 여러 교류에 대해 열린 자세를 지향합니다. 블로그에...,박찬엽,[],0
4,4,5,-1,5c66c39491b4130004e506c1,Materials,/material/,0000-12-30T00:00:00.000Z,"직접 제작한 자료를 공유합니다. 부족한 부분도 많고, 잘못된 부분이 있을 수도 있습...",박찬엽,[],0
...,...,...,...,...,...,...,...,...,...,...,...
35431,35431,35432,-1,5e43f1a9011afa5d389080b3,[ Springboot ] Unable to start embedded Tomcat...,https://brocess.tistory.com/262,2020-02-12T11:40:11.000Z,mac로컬 환경에서 가끔 스프링부트 어플리케이션 실행시 다음과 같은 에러가 발생한다...,김동범,['Spring'],0
35432,35432,35433,-1,5e43f1ac011afa5d389080b9,200211_TIL,https://velog.io/@hy9202/200211TIL,2020-02-12T11:41:40.000Z,200211_TIL 다시 공부한 것 HTTP 상태코드: HTTP 상태코드 | No...,홍영란,['Node'],0
35433,35433,35434,-1,5e43f1ac011afa5d389080b7,200212_TIL,https://velog.io/@hy9202/200212TIL,2020-02-12T11:46:54.000Z,200212_TIL CheckPoint #13 - SQL sql 1-to-ma...,홍영란,['SQL'],1
35434,35434,35435,-1,5e43f18b011afa5d389080af,[Windows] 윈도우 10 하드디스크 파티션 나누기,https://coding-factory.tistory.com/492,2020-02-12T12:01:10.000Z,.adsense_1 { display: inline-b...,우정태,['Windows'],1


### Convert to TSV
- title, description, tag만 기록
- title이나 description이 3글자 미만인 경우 제외
- label 컬럼 추가(default 1)
 - 0 : IT 관련 글 아님
 - 1 : 완전 IT 글
 - 2 : 애매함..(향후 카테고리를 더 나누기 위한 글 : 알고리즘 / 조직, 문화 / 디자인)

In [7]:
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

In [88]:
with open(POSTS_JSON_PATH, 'w') as outfile:
    json.dump(posts, outfile)

In [112]:
with open(POSTS_TSV_PATH, 'w') as f:
    # title, description, tags만 수집
    f.write('\t'.join(['label', 'title', 'description', 'tags', 'link']))
    f.write('\n')
    
    for post in posts:
        values = []
        if len(post['title']) < 3: continue
        if len(post['description']) < 3: continue
        if len(post['link']) < 10: continue
        title = re.sub('\t', '', post['title'])
        description = re.sub('\t', '', post['description'])
        tags = re.sub('\t', '', ' '.join(post['tags']))
        link = re.sub('\t', '', post['link'])
        f.write('\t'.join(['1', title, description, tags, link]))
        f.write('\n')
    print('done')

done
